<span style="font-family:Trebuchet MS; font-size:2em;">Project 3 | NB2: Cleaning and Preprocessing</span>

Riley Robertson | Reddit Classification Project | Market Research: Sports Fans in the U.S. and England

# **Imports and setup**

## Module Imports

I began my process by importing basic libraries and as I cleaned, I returned to add modules as necessary. I also set preferences, assigned variables, imported my data, and set up my main dataframe so I could begin cleaning.

In [1]:
# basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string

# custom
import utilities.densmore as dns

# date and time 
import datetime as dt
import time

# for CVEC test
from sklearn.feature_extraction.text import CountVectorizer

---

## Data Import and Setup

In [2]:
df_nfl = pd.read_csv('../data/1_raw/raw_nfl_v4.csv', low_memory=False)
df_epl = pd.read_csv('../data/1_raw/raw_epl_v4.csv', low_memory=False)

In [3]:
df_nfl.shape, df_epl.shape

((99661, 13), (99589, 13))

### Merging the DataFrames

In [4]:
df = pd.concat([df_epl, df_nfl], ignore_index=True)

### Checks

In [5]:
# df.shape
# df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199250 entries, 0 to 199249
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   index            199250 non-null  int64 
 1   subreddit        199250 non-null  object
 2   created_utc      199250 non-null  int64 
 3   author           199250 non-null  object
 4   num_comments     199250 non-null  int64 
 5   score            199250 non-null  int64 
 6   is_self          199250 non-null  bool  
 7   link_flair_text  29684 non-null   object
 8   title            199250 non-null  object
 9   selftext         168312 non-null  object
 10  full_link        199250 non-null  object
 11  date             199250 non-null  object
 12  time             199250 non-null  object
dtypes: bool(1), int64(4), object(8)
memory usage: 18.4+ MB


### Renaming PremierLeague Subreddit to 'epl'

In [6]:
df['subreddit'].value_counts()

nfl              99661
PremierLeague    99589
Name: subreddit, dtype: int64

In [7]:
df['subreddit'] = df['subreddit'].map(lambda x: 'epl' if x == 'PremierLeague' else x)

In [8]:
df['subreddit'].value_counts()

nfl    99661
epl    99589
Name: subreddit, dtype: int64

### Column Sorting and Filtering

As I cleaned, I realized that there were some columns that I didn't ultimately need, so I filtered out some of the columns that I initially included in my scraped data and resorted the remaining columns for ease of viewing. 

In [9]:
df = df[['subreddit', 'created_utc', 'date', 'time', 'link_flair_text', 'author', 'score', 'num_comments', 'index',  'title', 'selftext']]

In [10]:
# df.info()

In [11]:
df['subreddit'].value_counts()

nfl    99661
epl    99589
Name: subreddit, dtype: int64

As shown by the value counts above, I'm starting out with about 100,000 posts for each subreddit. I originally started with fewer, but after I began cleaning, I was quickly running of posts that that had the conditions I wanted. I returned to my Data Collection notebook and increased the number of posts to request from the API so that I'd begin my cleaning with a much greater volume of posts than I would eventually need. That way, I could be more decisive in dropping rows rather than trying to salvage content from posts that had incomplete or low quality information.

# **Basic Cleaning**

### Nulls

Nulls only exist in two columns: `link_flair_text` and `selftext`. 

I knew I had enough data that I could drop all of the posts with empty `selftext` fields, but I didn't want to lose the posts without tags (there are many). So I put 'none' into the `link_flair_text` fields and removed all rows with nulls after that, which left about 80,000 posts per subreddit.

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199250 entries, 0 to 199249
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   subreddit        199250 non-null  object
 1   created_utc      199250 non-null  int64 
 2   date             199250 non-null  object
 3   time             199250 non-null  object
 4   link_flair_text  29684 non-null   object
 5   author           199250 non-null  object
 6   score            199250 non-null  int64 
 7   num_comments     199250 non-null  int64 
 8   index            199250 non-null  int64 
 9   title            199250 non-null  object
 10  selftext         168312 non-null  object
dtypes: int64(4), object(7)
memory usage: 16.7+ MB


In [13]:
# fill nulls in link_flair_text column
df['link_flair_text'].fillna('none', inplace=True)

# remove all rows with nulls in selftext column
df.dropna(inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168312 entries, 0 to 199248
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   subreddit        168312 non-null  object
 1   created_utc      168312 non-null  int64 
 2   date             168312 non-null  object
 3   time             168312 non-null  object
 4   link_flair_text  168312 non-null  object
 5   author           168312 non-null  object
 6   score            168312 non-null  int64 
 7   num_comments     168312 non-null  int64 
 8   index            168312 non-null  int64 
 9   title            168312 non-null  object
 10  selftext         168312 non-null  object
dtypes: int64(4), object(7)
memory usage: 15.4+ MB


In [14]:
df['subreddit'].value_counts()

epl    86351
nfl    81961
Name: subreddit, dtype: int64

### Simple Duplicates

Dropping duplicates brings down PremierLeague posts to a good range, but the number of NFL posts is still much greater than necessary. As I move forward, I'll work on bringing down the number of NFL posts to at least roughly match that of the PremierLeague posts.

In [15]:
df.drop_duplicates(subset=['title'], inplace=True)
df.shape

(59730, 11)

In [16]:
df['subreddit'].value_counts()

nfl    53173
epl     6557
Name: subreddit, dtype: int64

In [17]:
df.drop_duplicates(subset=['selftext'], inplace=True)
df.shape

(56178, 11)

In [18]:
df['subreddit'].value_counts()

nfl    49832
epl     6346
Name: subreddit, dtype: int64

### Posts with deleted body text (selftext)

In [19]:
df.drop(axis=0, 
        labels=df[df['selftext'].str.startswith('[deleted]')].index, # Submissions with deleted selftext
        inplace=True)

df['subreddit'].value_counts()

nfl    49831
epl     6324
Name: subreddit, dtype: int64

### Posts with Markdown tables

In [20]:
markdowns = df[df['selftext'].str.contains('\|')]

In [21]:
markdowns['subreddit'].value_counts()

nfl    3906
epl     201
Name: subreddit, dtype: int64

In [22]:
df.drop(axis=0, labels=markdowns.index, inplace=True)

# **Deep Cleaning**

## Repeated Post Titles (Complex Duplicates)

### Overview

In [23]:
# df['selftext'].value_counts()[:30]
# df.head()

After looking at the value counts for the 'selftext' field, I realized that there were a lot of posts that the same, or very slightly different body text. But they weren't caught by the code to remove duplicates above. So I used the following code to look at the top 20 most common titles and there were quite a few that were re-used many times. 

In [25]:
# df['title'].value_counts()[:20]

Readable results from above code:

| Post Title                                         | Count | ┃ | Post Title                          | Count | ┃ | Post Title                                   | Count |
|:---------------------------------------------------|:------|:-:|:------------------------------------|:------|:-:|:---------------------------------------------|:------|
| Shitpost Saturday                                  | 174   | ┃ | Talko Tuesday                       | 124   | ┃ | r/PremierLeague Midweek Musings              | 13    |
| Water Cooler Wednesday                             | 159   | ┃ | r/PremierLeague Daily Discussion    | 71    | ┃ | Weekly /r/PremierLeague Subreddit Suggestion | 11    |
| Free Talk Friday                                   | 158   | ┃ | This Week's Top /r/NFL [Highlight]s | 22    | ┃ | Test                                         | 11    |
| Sunday Brunch                                      | 157   | ┃ | Weekend Wrap Up                     | 21    | ┃ | Daily Open Discussion Thread                 | 11    |
| Thursday Talk Thread... Yes That's The Thread Name | 141   | ┃ | Question                            | 15    | ┃ | Weekly Transfer Discussion Thread            | 8     |
| Weekend Wrapup                                     | 130   | ┃ | NFL Power Rankings (Combined)       | 15    | ┃ | test                                         | 7     |

They fell into several categories:
1. Open threads meant for discussion of topics of any kind, even if unrelated to the topic of the subreddit.
2. Discussion threads in which the topics might be related, but all of the content is in the comments rather than the body of the post
3. Posts with code and/or little-to-no useful content
4. Commonly used titles by different users to introduce a topic-relevant post

### Removing the posts

In [26]:
repeat_titles = ["Shitpost Saturday", "Water Cooler Wednesday", "Free Talk Friday", "Sunday Brunch", 
                 "Thursday Talk Thread... Yes That's The Thread Name", "Weekend Wrapup", "Talko Tuesday",
                 "r/PremierLeague Daily Discussion", "This Week's Top /r/NFL [Highlight]s", 
                 "Weekend Wrap Up", "NFL Power Rankings (Combined)", "r/PremierLeague Midweek Musings", 
                 "Whose Line is it Anyways Wednesday--Offseason Edition", 
                 "Weekly /r/PremierLeague Subreddit Suggestion", "Test", "Daily Open Discussion Thread",
                 "Weekly Transfer Discussion Thread", "test", "Your Weekly /r/nfl Recap", 
                 "NFL Power Rankings (Combined) Week 0",
                 "Should Ole stay at Manchester United or not? If he got sacked by the board, who will be the best replacement. Comment your thoughts below"
                ]

In [27]:
for title in repeat_titles:
    title_df = df[df['title'] == title]  
    df.drop(axis=0, labels=title_df.index, inplace=True)

In [28]:
df['subreddit'].value_counts()

nfl    45921
epl     6117
Name: subreddit, dtype: int64

## PremierLeague Poll Posts

I found 91 posts from the PremierLeague subreddit that contained a lot of unnecessary information and formatting was such that vectorizing would be significantly more complicated. I decided to simply remove them for simplicity.

In [29]:
len('[View Poll](https://www.reddit.com/poll/g437k5)')

47

In [30]:
poll_posts = df[df['selftext'].str.startswith('  [View Poll]')]

In [31]:
poll_posts.shape

(0, 11)

In [32]:
df.drop(axis=0, labels=poll_posts.index, inplace=True)

In [33]:
df['subreddit'].value_counts()

nfl    45921
epl     6117
Name: subreddit, dtype: int64

## PremierLeague Match Threads

I found 91 posts from the PremierLeague subreddit that contained a lot of unnecessary information and formatting was such that vectorizing would be significantly more complicated. I decided to simply remove them for simplicity.

In [34]:
match_thread_titles = ('[Match Thread]', 
                       '[Match thread]', 
                       '[match Thread]', 
                       '[match thread]')

In [35]:
df['title'].str.startswith(match_thread_titles).value_counts()

False    52029
True         9
Name: title, dtype: int64

In [36]:
df['title'].str.startswith(match_thread_titles).value_counts()

False    52029
True         9
Name: title, dtype: int64

In [37]:
match_threads = df[df['title'].str.startswith(match_thread_titles)]

In [38]:
match_threads.shape

(9, 11)

In [39]:
df['subreddit'].value_counts()

nfl    45921
epl     6117
Name: subreddit, dtype: int64

## Removing NFL posts with Tags

Game Thread, Serious, Look Here!, and others that are mostly comment threads of unrelated topics.

In [40]:
df_nfl['link_flair_text'].value_counts()[:10]

Look Here!                        1805
Game Thread                       1061
Serious                            748
Free Talk                          492
Free talk                          437
Removed: Rule 2 - Invalid Post     116
Post Game Thread                    72
Trash Talk                          60
Look Here                           51
game                                42
Name: link_flair_text, dtype: int64

In [41]:
nfl_with_tags = df[(df['link_flair_text'] != 'none') & (df['subreddit'] == 'nfl')]
df.drop(axis=0, labels=nfl_with_tags.index, inplace=True)

In [42]:
df['link_flair_text'].value_counts()[:10]

none                       47228
Discussion                  1583
Question                     919
Poll                         564
:xpl: Premier League         246
News                          79
:mun: Manchester United       72
:liv: Liverpool               62
:ars: Arsenal                 57
:che: Chelsea                 53
Name: link_flair_text, dtype: int64

In [43]:
df.drop(axis=0, labels=df[(df['link_flair_text'] == 'Poll')].index, inplace=True)

df['subreddit'].value_counts()

nfl    45124
epl     5553
Name: subreddit, dtype: int64

## NFL Posts Filtered by String Length

In order to reduce the number of posts I had from the r/nfl, I decided to filter based on length.

First, I created a DataFrame that contained only nfl posts.

In [44]:
df_filtered = df[df['subreddit'] == 'nfl']

df_filtered.shape

(45124, 11)

I then created a second DataFrame that contained only the rows I want to keep (rows with post lengths between 500 and 1200 characters was where I landed after several tests until I got the count of NFL posts down to a similar number as that of the EPL posts.

Taking this slightly roundabout way allowed me to see the number of posts I'd have remaining once I removed the excess from the main DataFrame.

In [45]:
df_lengthlimits = df[(df['selftext'].str.len()>500) & \
                     (df['selftext'].str.len()<1200) & \
                     (df['subreddit'] == 'nfl')]
df_lengthlimits.shape

(7245, 11)

Using the index of that second DataFrame, I removed all the posts I want to keep from the DataFrame I created above: 'df_filtered', thus giving me a DataFrame containing all of the posts I want to exclude. 

In [46]:
df_filtered = df_filtered.drop(axis=0, labels=df_lengthlimits.index)

In [47]:
df_filtered.shape

(37879, 11)

With that filtered DataFrame, I was able to use its index to drop all of the unwanted posts from the primary DataFrame.

In [48]:
df.drop(axis=0, labels=df_filtered.index, inplace=True)

In [49]:
df['subreddit'].value_counts()

nfl    7245
epl    5553
Name: subreddit, dtype: int64

Later on, I realized that there some extremely long posts (upwards of 25,000-30,000 characters) on the epl page that really skewed my distributions in the EDA section. I came back to remove those outliers and then move forward again from here.

The content is valuable, though, so I didn't want to trim too much. rather than trimming as far as a max of 1200 characters like I did for the NFL posts, I cut it off at 3,000. The distribution will still be off, but not nearly to the severe degree it was before.

In [50]:
df[(df['subreddit'] == 'epl') & (df['selftext'].str.len()>3000)].shape

(127, 11)

In [51]:
df = df[df['selftext'].str.len()<3000]

In [52]:
df.shape

(12671, 11)

In [53]:
df['subreddit'].value_counts()

nfl    7245
epl    5426
Name: subreddit, dtype: int64

# Re-Indexing

After removing so many rows, the DataFrame's index had gaps in its sequencing, so I decided to reset it to clean it up.

In [54]:
df[5:9]

,subreddit,created_utc,date,time,link_flair_text,author,score,num_comments,index,title,selftext
7,epl,1619271770,2021-04-24,06:42:50,Discussion,CC-33,1,3,7,My thoughts and prayers are with Jurgen Klopp ...,"Imagine being Jurgen Klopp right now, arguably..."
9,epl,1619278607,2021-04-24,08:36:47,Discussion,Cheerful_Jerry9603,1,2,9,Premier League Players who should finish off t...,Chinese Super League is known to be the last p...
14,epl,1619283368,2021-04-24,09:56:08,Question,alphaftw1,1,8,14,"Hypothetical situation, what happens if both c...","So let’s say this season, arsenal win the euro..."
15,epl,1619283692,2021-04-24,10:01:32,Question,imjonathvn,1,24,15,"Norwich, Watford, and Bournemouth might all ge...",Norwich and Watford have already been promoted...


The 'index' column can serve as a record of each posts original index number in case it's ever needed going forward.

In [55]:
df.reset_index(drop=True, inplace=True)

In [56]:
df[5:9]

,subreddit,created_utc,date,time,link_flair_text,author,score,num_comments,index,title,selftext
5,epl,1619271770,2021-04-24,06:42:50,Discussion,CC-33,1,3,7,My thoughts and prayers are with Jurgen Klopp ...,"Imagine being Jurgen Klopp right now, arguably..."
6,epl,1619278607,2021-04-24,08:36:47,Discussion,Cheerful_Jerry9603,1,2,9,Premier League Players who should finish off t...,Chinese Super League is known to be the last p...
7,epl,1619283368,2021-04-24,09:56:08,Question,alphaftw1,1,8,14,"Hypothetical situation, what happens if both c...","So let’s say this season, arsenal win the euro..."
8,epl,1619283692,2021-04-24,10:01:32,Question,imjonathvn,1,24,15,"Norwich, Watford, and Bournemouth might all ge...",Norwich and Watford have already been promoted...


# **Column Clean-up**

## Renaming 'selftext' to 'post'

I renamed this column because it felt more intuitive and it's shorter/easier to type.

In [57]:
df['post'] = df['selftext']

In [58]:
df.drop(columns='selftext', inplace=True)

## Merging 'title' and 'post' into an 'alltext' column

I decided to merge title and post into a single column so that I could keep all relevant text that I'd be using for modeling together in a single field. And any additional changes I made to the text could be made to that column only. 

In [59]:
df['alltext'] = df['title'] + ' ' + df['post']

In [60]:
# checks
# pd.DataFrame(df.iloc[343]).T[['title', 'post', 'alltext']]

In [61]:
len(pd.DataFrame(df.iloc[343]).T['title'][343]) + len(pd.DataFrame(df.iloc[343]).T['post'][343]), \
len(pd.DataFrame(df.iloc[343]).T['alltext'][343])


(821, 822)

## Renaming 'num_comments' to 'comments'

Shortening column name

In [62]:
df['comments'] = df['num_comments']

df.drop(columns='num_comments', inplace=True)

## Renaming 'link_flair_text' to 'tag'

In [63]:
df['tag'] = df['link_flair_text']

df.drop(columns='link_flair_text', inplace=True)

## Creating 'target' column

Here I created a column that represents each row's subreddits as a 1 or 0, which will allow our models to easily recognize and interact with the data.

In [64]:
df['target'] = df['subreddit'].map(lambda x: 1 if x == 'nfl' else 0)

## Re-ordering columns

Old Order:

'subreddit',  
'created_utc', 'date', 'time',  
'link_flair_text', 'author', 'score', 'num_comments',  
'index',  'title', 'selftext'


In [65]:
df = df[[
        'subreddit',
         'target', 
         'author', 
         'score', 
         'comments', 
         'tag', 
         'index',
         'created_utc', 
         'date', 
         'time',
         'title', 
         'post', 
         'alltext'
        ]]

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12671 entries, 0 to 12670
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subreddit    12671 non-null  object
 1   target       12671 non-null  int64 
 2   author       12671 non-null  object
 3   score        12671 non-null  int64 
 4   comments     12671 non-null  int64 
 5   tag          12671 non-null  object
 6   index        12671 non-null  int64 
 7   created_utc  12671 non-null  int64 
 8   date         12671 non-null  object
 9   time         12671 non-null  object
 10  title        12671 non-null  object
 11  post         12671 non-null  object
 12  alltext      12671 non-null  object
dtypes: int64(5), object(8)
memory usage: 1.3+ MB


In [67]:
df['subreddit'].value_counts()

nfl    7245
epl    5426
Name: subreddit, dtype: int64

In [68]:
df['subreddit'].value_counts(normalize=True)

nfl    0.571778
epl    0.428222
Name: subreddit, dtype: float64

In [69]:
df[1984:1990]

,subreddit,target,author,score,comments,tag,index,created_utc,date,time,title,post,alltext
1984,epl,0,halfbaht,1,329,:brh: Brighton &amp; Hove Albion,10336,1601127114,2020-09-26,06:31:54,How unlucky can one side be?,How unlucky can one side be? Hit the post 5 ti...,How unlucky can one side be? How unlucky can o...
1985,epl,0,TeddyMMR,1,13,Discussion,10337,1601127570,2020-09-26,06:39:30,So if we can change decisions after the match ...,The refereeing in this match was horrendous. 2...,So if we can change decisions after the match ...
1986,epl,0,Chrisflev,1,3,Question,10338,1601128425,2020-09-26,06:53:45,Full Player Game Stats,Does anyone know of a site or app that gives y...,Full Player Game Stats Does anyone know of a s...
1987,epl,0,TooSpursy,1,8,:tot: Tottenham Hotspur,10339,1601129611,2020-09-26,07:13:31,Where to watch PL in the U.S.?,If you live in the U.S. where is the best plac...,Where to watch PL in the U.S.? If you live in ...
1988,epl,0,entertainak47,1,8,Question,10340,1601130904,2020-09-26,07:35:04,Brighton-UTD Penalty:,In the Brighton-UTD game the ref game UTD a pe...,Brighton-UTD Penalty: In the Brighton-UTD game...
1989,epl,0,gl6ry,1,6,Question,10341,1601131905,2020-09-26,07:51:45,Which team has the coolest looking jerseys in ...,I’m new to this sport as a fan and trying to d...,Which team has the coolest looking jerseys in ...


---

# **Final Cleaning**

Once columns were cleaned up and the `'alltext'` column was made (from `'title'` and `'selftext'`), I did one more pass over the new column to remove specific strings of text.

## Subreddit name strings

The names of the subreddits included in titles and body text are likely to be obvious tells for classification, which will be great for our model - helping to ensure high accuracy classification of posts for OverArmor. 

For EDA, however, removing them might be better, as it will give us a cleaner look at the common vernacular of each community.

In [70]:
titlecount_alltext_nfl = df[df['alltext'].str.contains('r/nfl')].shape[0] + df[df['alltext'].str.contains('r/NFL')].shape[0]
titlecount_alltext_epl = df[df['alltext'].str.contains('r/premierleague')].shape[0] + df[df['alltext'].str.contains('r/PremierLeague')].shape[0]

In [71]:
print(f"Count of 'r/nfl' in 'alltext' column: {titlecount_alltext_nfl}")
print(f"Count of 'r/PremierLeague' in 'alltext' column: {titlecount_alltext_epl}")

Count of 'r/nfl' in 'alltext' column: 321
Count of 'r/PremierLeague' in 'alltext' column: 41


## URLs, Punctuation, and Numbers

### Remove URLs

For simplicity, I removed all urls and punctuation without exception. This will help me get clean tokens when I get to the point of tokenizing and vectorizing for analysis.

I went through many iterations of different code to do it and ultimately got Devin's help and the code inside the 'remove_clutter' function is his.

In [72]:
def remove_clutter(text):
    return re.sub(r'http\S+', '', text).translate(str.maketrans('', '', string.punctuation)).strip()

In [73]:
test_sentence = 'https://stackoverflow.com is the best!! and... I learn\'ed a lot `from https://towardsdatascience.com'

remove_clutter(test_sentence)

'is the best and I learned a lot from'

In [74]:
for column in ['title', 'post', 'alltext']:
    df[column] = df[column].map(lambda x: remove_clutter(x))

# **Exports**

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12671 entries, 0 to 12670
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subreddit    12671 non-null  object
 1   target       12671 non-null  int64 
 2   author       12671 non-null  object
 3   score        12671 non-null  int64 
 4   comments     12671 non-null  int64 
 5   tag          12671 non-null  object
 6   index        12671 non-null  int64 
 7   created_utc  12671 non-null  int64 
 8   date         12671 non-null  object
 9   time         12671 non-null  object
 10  title        12671 non-null  object
 11  post         12671 non-null  object
 12  alltext      12671 non-null  object
dtypes: int64(5), object(8)
memory usage: 1.3+ MB


## Clean Export for EDA and Modeling

In [76]:
df.to_csv('../data/2_clean/reddit_posts_clean.csv', index=False)

In [77]:
# df.head(20)

In [78]:
# df[['title', 'post', 'alltext']].head(20)

Having collected and cleaned our data, I completed my delieverable for OverArmor's first request. 

It remains to be seen how my models will do, but based on the way the data looks, I expect decent results. I think there are strong enough differences between the language used in these subreddits that the model will be able to do a good job. 

Team names, city names, unique words 